In [24]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_API_KEY"] = os.getenv("HF_API_KEY")

In [25]:
from langchain_huggingface import HuggingFaceEmbeddings

vectorizer = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [26]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances


In [27]:
document=["This is a test sentence.",
          "This is another test sentence.",
          "Who is prime minister of india?"]

In [28]:
query = "Narendra modi is prime minister of india."

In [29]:
document_vectors = vectorizer.embed_documents(document)

In [30]:
query_vector = vectorizer.embed_query(query)

In [31]:
cosine_similarity([query_vector], document_vectors)


array([[0.13925151, 0.13833855, 0.73641668]])

In [32]:
euclidean_distances([query_vector], document_vectors)

array([[1.31205837, 1.31275396, 0.72606242]])

In [33]:
import faiss

ModuleNotFoundError: No module named 'faiss'